# This project we are slove the eight-puzzle

This assigment require 3 tpyes of search

1.  Uniform Cost Search

2.  A\* with the Misplaced Tile heuristic.

3.  A\* with the Euclidean Distance heuristic. 
   
   Goal state:

             1 2 3 

             4 5 6

             7 8 0

In [91]:
import numpy as np
import queue  # for priority queue
import math # for math fuctions
# this is 3 by 3 array store the goal state of the puzzle
# I used 0 to represent the blank space inside he puzzle
Goal_State = np.array([[1, 2, 3], 
                       [4, 5, 6], 
                       [7, 8, 0]]).reshape(3,3)
# test cases: 
# the 6 test cases are get from the "Project1-Report Guidelines" https://drive.google.com/file/d/1h0t0NGQQtI7Fl9B8WzlMLuCn-sFNu0RO/view
trival_case = np.array([[1, 2, 3], 
                        [4, 5, 6], 
                        [7, 8, 0]]).reshape(3,3)
very_easy_case = np.array([[1, 2, 3], 
                           [4, 5, 6], 
                           [7, 0, 8]]).reshape(3,3)
easy_case = np.array([[1, 2, 0], 
                      [4, 5, 3], 
                      [7, 8, 6]]).reshape(3,3)
doable_case = np.array([[0, 1, 2], 
                        [4, 5, 3], 
                        [7, 8, 6]]).reshape(3,3)
oh_boy_case = np.array([[8, 7, 1], 
                        [6, 0, 2], 
                        [5, 4, 3]]).reshape(3,3)
impossible_case = np.array([[1, 2, 3], 
                            [4, 5, 6], 
                            [8, 7, 0]]).reshape(3,3)
# two more cases
doable2_case = np.array([[3, 1, 2], 
                         [4, 5, 0], 
                         [7, 8, 6]]).reshape(3,3)

easy_case2 = np.array([[4, 1, 2], 
                       [7, 5, 0], 
                       [8, 6, 3]]).reshape(3,3)
visited_state = set([])
print("Goal State:")
print(*Goal_State, sep='\n')
vistied_states = set([]) # this is a list to store the visited states"

Goal State:
[1 2 3]
[4 5 6]
[7 8 0]


In [92]:
# for our tree node class: depth, cost, and puzzel's row and columns for the blank space
# how to construct a tree in python https://www.delftstack.com/howto/python/trees-in-python/

class Node:
    # initalize the instance
    # # parent of current node/state, each node may have a child/state(create by 4 operations), operation, 
    #      depth of the parent node
    # since our step cost is 1, we can use the same cost for each operation which it is the depth = step_cost (g(n))
    # path_cost(f(n)) = the step cost (g(n)) + heuristic cost (h(n))
    # for uniform cost search, we not use the heuristic cost fucntion,so h(n) = zero
    # for A* search, we use the heuristic cost fucntion and h(n) are the euclidean distance/misplaced tiles/manhattan distance
    def __init__(self, state, parent, operations, tree_depth, path_cost, heuristic_cost):
        # we have 4 type of operations to move the blank space, move up, down, left,and right.
        # so we have 4 possibles way to construct children nodes/state
        self.state = state # the puzzle state
        self.parent = parent # the parent node of current puzzle
        self.operations = operations # the operation to move the blank space: up, down, left, right
        self.depth = tree_depth # the depth of the puzzle or the cost from the root node to the current node
        self.path_cost = path_cost # the estimate cost from the root node to the current node
        self.heristic_cost = heuristic_cost # the heuristic cost calculate by the euclidean distance/misplaced tiles/manhattan distance
            
    def get_puzzle(self): # return the puzzle state
        return self.state
    def get_parent(self): # return the parent node
        return self.parent
    def get_operations(self): # return the operations
        return self.operations
    def get_depth(self): # return the depth of the node
        return self.depth
    def get_path_cost(self): # return the path cost
        return self.path_cost
    def get_heristics_cost(self): # return the heuristic cost
        return self.heristic_cost
    def __lt__(self, next): # this is for the priority queue
        return self.path_cost < next.path_cost

In [93]:
# copy the puzzle to a new puzzle without change the original puzzle
def copy_p(current_puzzle): # return the copy puzzle
  res = np.zeros(shape=(3,3), dtype=int) # create a new 3x3 array
  for n in range(len(current_puzzle)): # copy the puzzle to the new array by using for loop
    for m in range(len(current_puzzle[0])): 
      res[n][m] = current_puzzle[n][m] # copy the puzzle to the new array
  return res
# we have 4 operations for move the blank space: up, down, left, right
# this 4 function return 2 variables: 
#   first variables: True if it can be move (puzzle can be change). Otherwise, False if cannot be move (no change of the puzzle)
#   scecond varaibles: 3x3 array that represent the current puzzle(it change by one of the operations)
def move_up(current_puzzle):
  row, col = find_blank_space_loc(current_puzzle) # find the location of the blank space
  # return varibale: is_move and result puzzle
  is_move = None  
  result = copy_p(current_puzzle) # store the result puzzle, if we return the result puzzle as input: this indicate blank space cannot be move 
  # if the blank space at the first row of the puzzle cannot move up
  if (row == 0):
    is_move = False
  else: # 2nd and 3rd row can be move up
    # store the value that have exchange location with the blank space
    result[row][col] = result[row-1][col]
    # move up: store blank space one rows up
    result[row-1][col] = 0
    is_move = True
  return is_move, result
def move_down(current_puzzle):
  row, col = find_blank_space_loc(current_puzzle) # find the location of the blank space
  # return varibale: is_move and result puzzle
  is_move = None  
  result = copy_p(current_puzzle) 
  # if the blank space at the 3rd row of the puzzle cannot move down
  if (row == 2):
    is_move = False
  else: # 1st and 2nd row can be move down
    # store the value that have exchange location with the blank space
    result[row][col] = result[row+1][col]
    # move up: store blank space one row down
    result[row+1][col] = 0
    is_move = True
  return is_move, result
def move_left(current_puzzle):
  row, col = find_blank_space_loc(current_puzzle) # find the location of the blank space
  # return varibale: is_move and result puzzle
  is_move = None  
  result = copy_p(current_puzzle) 
  # if the blank space at the first column of the puzzle cannot move left
  if (col == 0):
    is_move = False
  else: # 2nd and 3rd columns can be move left
    # store the value that have exchange location with the blank space
    result[row][col] = result[row][col-1]
    # move up: store blank space one columns left
    result[row][col-1] = 0
    is_move = True
  return is_move, result
def move_right(current_puzzle):
  row, col = find_blank_space_loc(current_puzzle) # find the location of the blank space
  # return varibale: is_move and result puzzle
  is_move = None  
  result = copy_p(current_puzzle)  
  # if the blank space at the 3rd column of the puzzle cannot move right
  if (col == 2):
    is_move = False
  else: # 1st and 2nd column can be move right
  # store the value that have exchange location with the blank space
    result[row][col] = result[row][col+1]
    # move up: store blank space one columns right
    result[row][col+1] = 0
    is_move = True
  return is_move, result


# this function check for the puzzle current is hit the got stat or not
def is_goal_state(current_puzzle):
  # use python liberay function to compare our current puzzle with goal state
  return (np.array_equal(Goal_State,current_puzzle))

# for test is_goal_state function
#print(is_goal_state(Goal_State))
#print(is_goal_state(easy_case))
#print(is_goal_state(very_easy_case))
#print(is_goal_state(doable_case))
# this function finding the location of the blank space in the puzzle
def find_blank_space_loc(current_puzzle):
  # return varibles for row and columns of the blank space that is locate in the puzzle
  p_row = 0
  p_col = 0
  for n in range(len(current_puzzle)):
    for m in range(len(current_puzzle[0])):
      if (current_puzzle[n][m] == 0): # if we find the blank space locate
        # store the postion in our return varible
        p_row = n
        p_col = m
  return p_row, p_col

# for test find_blak_space_loc function
#row, col = find_blank_space_loc(test1_case)
#print(row)
#print(col)"

In [94]:
# test our 4 operations function
print("test our 4 operations function all move from Goal State:")
test = [[1,2,3],[4,0,5],[6,7,8]]
test1 = test
print(test[0])
print(test[1])
print(test[2])
print("move up")
r, c = find_blank_space_loc(test)
isMove, puzzle = move_up(test)
print(puzzle[0])
print(puzzle[1])
print(puzzle[2])
print("move down")
r, c = find_blank_space_loc(test)
isMove, puzzle = move_down(test)
print(puzzle[0])
print(puzzle[1])
print(puzzle[2])
print("move right")
r, c = find_blank_space_loc(test)
isMove, puzzle = move_right(test)
print(puzzle[0])
print(puzzle[1])
print(puzzle[2])
print("move left")
r, c = find_blank_space_loc(test)
isMove, puzzle = move_left(test)
print(puzzle[0])
print(puzzle[1])
print(puzzle[2])

test our 4 operations function all move from Goal State:
[1, 2, 3]
[4, 0, 5]
[6, 7, 8]
move up
[1 0 3]
[4 2 5]
[6 7 8]
move down
[1 2 3]
[4 7 5]
[6 0 8]
move right
[1 2 3]
[4 5 0]
[6 7 8]
move left
[1 2 3]
[0 4 5]
[6 7 8]


In [95]:
def misplaced_tile_h(state,goal_state):
    # we will return the number of misplaced tile
    # use python liberary function: this compare current puzzle stete with goal state
    if (np.sum(np.not_equal(state, goal_state))-1) < 0: # -1 because we dont need to count the blank space
        return 0 # if the current puzzle state is equal to goal state, return 0
    else:
        return (np.sum(np.not_equal(state, goal_state))-1) # if not equal, return the number of misplaced tile

In [96]:
# formula get it from https://stackoverflow.com/questions/16869920/a-heuristic-calculation-with-euclidean-distance
# this function calculate the euclidean distance between the current state and goal state
def euclidean_distance_h(state,goal_state):
    # we will return the euclidean distance between the current state and goal state
    cost = 0
    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] != 0: # if the current state is not the blank space
                # ((x1- x2)^2 + (y1 - y2)^2)^(1/2)
                cost += math.sqrt( (i - (state[i][j] - 1) // 3)**2 + (j - (state[i][j] - 1) % 3)**2) # calculate the cost
    return cost # return the cost

In [97]:
# Manhattan distance heuristic not require for this assignment，I just want to test if this is better than misplaced tile heuristic and euclidean distance heuristic
def manhattan_distance_h(state,goal_state):
    cost = 0 # cost of the current state
    for i in range(len(state)): # loop for row
        for j in range(len(state[0])): # loop for column
            if state[i][j] != 0: # if the current state is not the blank space
                # abs(x1- x2)^2 + abs(y1 - y2)^2
                cost += abs(i - (state[i][j] - 1) // 3) + abs(j - (state[i][j] - 1) % 3) # calculate the cost
    return cost

In [98]:
print("test our heristic function with the 6 test case:")
print("test our misplaced_tile_h function:")
print(misplaced_tile_h(trival_case,Goal_State))
print(misplaced_tile_h(very_easy_case,Goal_State))
print(misplaced_tile_h(easy_case,Goal_State))
print(misplaced_tile_h(doable_case,Goal_State))
print(misplaced_tile_h(oh_boy_case,Goal_State))
print(misplaced_tile_h(impossible_case,Goal_State))
print()
print("test our euclidean_distance_h function:")
print(euclidean_distance_h(trival_case,Goal_State))
print(euclidean_distance_h(very_easy_case,Goal_State))
print(euclidean_distance_h(easy_case,Goal_State))
print(euclidean_distance_h(doable_case,Goal_State))
print(euclidean_distance_h(oh_boy_case,Goal_State))
print(euclidean_distance_h(impossible_case,Goal_State))
print()
print("test our manhattan_distance_h function:")
print(manhattan_distance_h(trival_case,Goal_State))
print(manhattan_distance_h(very_easy_case,Goal_State))
print(manhattan_distance_h(easy_case,Goal_State))
print(manhattan_distance_h(doable_case,Goal_State))
print(manhattan_distance_h(oh_boy_case,Goal_State))
print(manhattan_distance_h(impossible_case,Goal_State))

test our heristic function with the 6 test case:
test our misplaced_tile_h function:
0
1
2
4
8
1

test our euclidean_distance_h function:
0.0
1.0
2.0
4.0
14.714776642118867
2.0

test our manhattan_distance_h function:
0
1
2
4
18
2


In [99]:
# user choice for type of algorithm
def get_heristics(puzzle, options):
    if (options == 0): # uniform cost search
        return 0
    elif (options == 1): # misplaced tiles fucntion
        return misplaced_tile_h(puzzle,Goal_State)
    elif (options == 2): # euclidean distance function
        return euclidean_distance_h(puzzle,Goal_State)
    elif  (options == 3): # manhattan distance function
        return manhattan_distance_h(puzzle,Goal_State)

In [100]:
def expand_node(node, alogorithm):
    options = alogorithm # 0: uniform cost search, 1: misplaced tiles fucntion, 2: euclidean distance function 3: manhattan distance function
    child_node = [] # list of child node
    if node == None: # if the node is none
        return False # return false
    else: # if the node is not none
        # call 4 operations fucntion to check how current puzzle can move
        isUp, up_puzzle = move_up((node.get_puzzle())) # move up
        isDown, down_puzzle = move_down((node.get_puzzle())) # move down
        isRight, right_puzzle = move_right((node.get_puzzle())) # move right
        isLeft, left_puzzle = move_left((node.get_puzzle())) # move left
        
        # if the puzzle can move up then add it to the queue
        if (isUp == True):
            #print("move up")
            if (tuple(up_puzzle.reshape(1,9)[0]) not in vistied_states): # if the puzzle is not visited yet
                vistied_states.add(tuple(up_puzzle.reshape(1,9)[0])) # add the puzzle to the visited list
                #print("not visited yet")
                h = get_heristics(up_puzzle,options) # get the heristics of the puzzle
                # create a new node with the new puzzle
                children = Node(up_puzzle, node.get_puzzle(), "up", node.get_depth() + 1, node.get_depth() + h, h) # create a new node with the new puzzle  and the parent node
                child_node.append(children) # add the new node to the queue
                #q.put(children) # add the new node to the queue
        if (isDown == True): # if the puzzle can move down then add it to the queue
            #print("move down")
            if (tuple(down_puzzle.reshape(1,9)[0]) not in vistied_states): # if the puzzle is not in the visited list
                #print("not visited")
                vistied_states.add(tuple(down_puzzle.reshape(1,9)[0]) ) # add the puzzle to the visited list
                h = get_heristics(down_puzzle,options) # get the heristics of the puzzle
                children = Node(down_puzzle, node.get_puzzle(), "down", node.get_depth() + 1, node.get_depth() + h, h) # create a new node with the new puzzle
                child_node.append(children) # add the new node to the queue
                #q.put(children) # add the new node to the queue
        if (isRight == True):  #    if the puzzle can move right then add it to the queue
            #print("move right")
            if (tuple(right_puzzle.reshape(1,9)[0])  not in vistied_states): # if the puzzle is not in the visited list
                #print("not visited")
                vistied_states.add(tuple(right_puzzle.reshape(1,9)[0])) # add the puzzle to the visited list
                h = get_heristics(right_puzzle,options) # get the heristics of the puzzle
                children = Node(right_puzzle, node.get_puzzle(), "right",node.get_depth() + 1, node.get_depth() + h, h) # create a new node with the new puzzle
                child_node.append(children) # add the new node to the queue
                #q.put(children) # add the new node to the queue
        if (isLeft == True): # if the puzzle can move left then add it to the queue
            #print("move left")
            if (tuple(left_puzzle.reshape(1,9)[0]) not in vistied_states): 
                #print("not in visited list")
                vistied_states.add(tuple(left_puzzle.reshape(1,9)[0])) # add the puzzle to the visited list
                h = get_heristics(left_puzzle,options) # get the heristics of the puzzle
                children = Node(left_puzzle, node.get_puzzle(), "left",node.get_depth() + 1, node.get_depth() + h, h) # create a new node with the new puzzle
                child_node.append(children)# add the new node to the queue
                #q.put(children) # add the new node to the queue
    return child_node # return the list of child node

In [101]:
#root = Node(very_easy_case, None, None, 0, 0)
#q = queue.PriorityQueue()
#q.put(root)
# use tuple，it show error when I use array or list，because the array and list are mutable
# So, I change my 3x3 array to a tuple 1x9 array
#vistied_states.add(tuple((root.state).reshape(1,9)[0])) # add the puzzle to the visited list
#x = expand_node(root)
#for i in range(len(x)):
 #     q.put(x[i])

#a = q.get()
#print(a.get_puzzle(), a.get_operations(),a.get_depth())
#a = q.get()
#print(a.get_puzzle(), a.get_operations(),a.get_depth())
#a = q.get()
#print(a.get_puzzle(), a.get_operations(),a.get_depth())
#a = q.get()
#print(a.get_puzzle(), a.get_operations(),a.get_depth())\n"

In [102]:
def type_search(puzzle,alg): # alg = 0: uniform cost search, 1: misplaced tiles fucntion, 2: euclidean distance function, 3: manhattan distance function
    result_q = queue.PriorityQueue() # create a priority queue to store the result
    initial = copy_p(puzzle) # create a copy of the puzzle
    q = queue.PriorityQueue() # create a priority queue to store the nodes
    root = Node(initial, None, None, 0, 0+get_heristics(initial, alg), get_heristics(initial, alg)) # create a root node
    q.put(root) # add the root node to the queue
    # it need to use tuple，it show error when I use array or list，because the array and list are mutable
    # So, I change my 3x3 array to a tuple 1x9 array
    vistied_states.add(tuple((initial).reshape(1,9)[0])) #  add the puzzle to the visited list
    node_pop = 0 # the number of nodes poped from the queue (same as the number of nodes store in the result queue)
    max_queue = 0 # the max number of nodes in the result queue
    max_queue_temp = 0 # for the max number of nodes in the temp queue
    num_nodes = 0 # the number of nodes expanded
    print("start searching.......")
    while not q.empty(): # while the queue is not empty
        #print("check: ", node_pop) 
        node_pop+=1
        #print("queue size: ", q.qsize())
        if q.qsize() > max_queue_temp: # for store the max number of nodes in the  temp queue
            max_queue_temp = q.qsize()
        #print("queue size: ", max_queue)
        node = q.get() # get the node from the queue
        result_q.put(node) # add the node to the result queue
        print(*node.get_puzzle(), sep = "\n")
        print("operations = " , node.get_operations())
        print("depth(g(n)) = " ,   node.get_depth())
        print("h(n) = " , node.get_heristics_cost())
        print("f(n) = " , node.get_depth() + node.get_heristics_cost())
        print("Expanded nodes = ", num_nodes, ". current queue size = ", result_q.qsize()) # uncomment this line to see the number of nodes expanded if the case failed
        
        if result_q.qsize() > max_queue: # for store the max number of nodes in the result queue
            max_queue = result_q.qsize()
        if is_goal_state(node.get_puzzle()): # if the current state is the goal state
            #while not result_q.empty():
            #    a = result_q.get()
            #    print(a.get_puzzle())
            #    print("operations = " , a.get_operations())
            #    print( "depth = " ,   a.get_depth())
            #    path.extend(a.get_puzzle(), a.get_operations(), a.get_depth())
            
            # print the result
            print("Goal state found!")
            #print(*node.get_puzzle(), sep = "\n")
            print("To solve this problem, the number of nodes expand is: ", num_nodes)
            print("The maximum number of nodes in the queue at any one time is: ", max_queue)
            print("The depth of the goal state is: ", node.get_depth())
            return True
        elif is_goal_state(node.get_puzzle()) == False: # if the current state is not the goal state
            # expand the node
            children = expand_node(node, alg) # get the children of the current node
            num_nodes += len(children) # add the number of nodes expanded
            print("Expanding the nodes..........")
            print("Expand", len(children), "nodes")
            print()
            #print(len(children))
            for i in range(len(children)): # for each child node
                q.put(children[i]) # add the child node to the queue
                vistied_states.add(tuple((children[i].get_puzzle()).reshape(1,9)[0])) # add the puzzle to the visited list
    # the goal state is not found and end of program   
    print("Goal state not found!")
    print("For solve this problem, the number of nodes expand is: ", num_nodes)
    print("The maximum number of nodes in the queue at any one time is: ", max_queue)
    print("The depth of the goal state is: ", node.get_depth())
    #return False
    
    return False

In [103]:
# uncomment the following line to test the uniform cost search for the test case, uncoment one of the lines below to test and restore the program for every test case
#type_search(trival_case, 0)
#type_search(very_easy_case, 0)
#type_search(easy_case, 0)
#type_search(easy_case2, 0)
#type_search(doable_case, 0)
#type_search(doable2_case, 0)
#type_search(oh_boy_case, 0)
#type_search(impossible_case, 0)

# uncoment the following line to test the A* with misplaced tiles 
#type_search(trival_case, 1)
#type_search(very_easy_case, 1)
#type_search(easy_case, 1)
#type_search(easy_case2, 1)
#type_search(doable_case, 1)
#type_search(doable2_case, 1)
#type_search(oh_boy_case, 1)
#type_search(impossible_case, 1)

# uncoment the following line to test the A* with euclidean distance 
#type_search(trival_case, 2)
#type_search(very_easy_case, 2)
#type_search(easy_case, 2)
#type_search(easy_case2, 2)
#type_search(doable_case, 2)
#type_search(doable2_case, 2)
#type_search(oh_boy_case, 2)
#type_search(impossible_case, 2)

# optional: uncoment the following line to test the A* with euclidean distance 
#type_search(trival_case, 3)
#type_search(very_easy_case, 3)
#type_search(easy_case, 3)
#type_search(easy_case2, 3)
#type_search(doable_case, 3)
#type_search(doable2_case, 3)
#type_search(oh_boy_case, 3)
#type_search(impossible_case, 3)


In [104]:
# main function ask user input
def main():
    type_case = None # for store user's input of choice of test cases
    first_row = None # for store user's input of first row of the puzzle
    second_row = None # for store user's input of 2nd row of the puzzle
    user_puzzle = None # for store user's input of puzzle
    third_row = None # for store user's input of 3rd row of the puzzle
    choice = None # for store chocie of alg
    initial_puzzle = None # for store usr's choice of type of puzzle to begin with it
    # display msg and ask user input
    print("welcome to 862039649 8 puzzle solver.")
    type_puzzle = input("Type \"1\" to use a defualt puzzle, or \"2\" to enter your own puzzle.")
    #print(type_puzzle)
    # ask uer to chose type of puzzle in default
    if (int(type_puzzle) == 1):
      print("Chose the following defualt puzzle(diffculty from easy to harder)")
      print("Enter 1 to chose trival")
      print("Enter 2 to chose very easy")
      print("Enter 3 to chose easy")
      print("Enter 4 to chose easy2")
      print("Enter 5 to chose doable")
      print("Enter 6 to chose doable2")
      print("Enter 7 to chose oh boy")
      print("Enter 8 to chose impossible")
      type_case = input("Enter your choice:")
      if (int(type_case) == 1):
          initial_puzzle = copy_p(trival_case)
          #print(1)
      elif (int(type_case)  == 2):
          initial_puzzle = copy_p(very_easy_case)
          #print(2)
      elif (int(type_case)  == 3):
          initial_puzzle = copy_p(easy_case)
          #print(3)
      elif (int(type_case)  == 4):
          initial_puzzle = copy_p(easy_case2)
          #print(4)
      elif (int(type_case)  == 5):
          initial_puzzle = copy_p(doable_case)
          #print(5)
      elif (int(type_case)  == 6):
          initial_puzzle = copy_p(doable2_case) 
          #print(6) 
      elif (int(type_case)  == 7):
          initial_puzzle = copy_p(oh_boy_case) 
          #print(7) 
      elif (int(type_case)  == 8):
          initial_puzzle = copy_p(impossible_case) 
          #print(8) 
    # use chose to input own puzzle
    elif (int(type_puzzle) == 2):
        print("Enter your pullze, use a zero to represent the blank(vaild # 0-8)")
        first_row = input("Enter the first row, use space or tabs between numbers:")
        second_row = input("Enter the second row, use space or tabs betweeen numbers:")
        third_row = input("Enter the third row, use space or tabls between numbers:")
        first_row = [int(i) for i in first_row.split()] # split the string to a int list
        second_row = [int(i) for i in second_row.split()] # split the string to a int list
        third_row = [int(i) for i in third_row.split()] # split the string to a int list
        user_puzzle = np.array([first_row, second_row, third_row]).reshape(3,3) #   create a puzzle from the input
        initial_puzzle = copy_p(user_puzzle) # copy the user puzzle to the initial puzzle
        
    print()
    print("The choice of algorithm")
    print("Enter 1 for Uniform Cost Search")
    print("Enter 2 for A* with the Misplaced Tile heurstic")
    print("Enter 3 for A* with the Eucledian distance heurstic")
    print("Options: Enter 4 for A* with the ma heurstic")
    choice = input("Enter your choice of algorithm:")
    print()

    # find the row and column of blank space that locate in user's input puzzle or test puzzle
    #row, col = find_blank_space_loc(easy_case)
    # create priority queue instance
    #puzzle_queue = queue.PriorityQueue()
    # for store the size of the priorty queue
    #queue_size = 0
    ## for store # of node that expend
    #node_expend = 0

    # choice 1 of uniform cost search
    if (int(choice) == 1):
        type_search(initial_puzzle, 0)
    elif (int(choice) == 2): # choice 2 of A* with the Misplaced Tile heurstic
         type_search(initial_puzzle, 1)
    elif (int(choice) == 3): # choice 3 of A* with the Eucledian distance heurstic
        type_search(initial_puzzle, 2)
    elif (int(choice) == 4): # choice 4 of A* with the manhattan distance heurstic
        type_search(initial_puzzle, 3)
        

In [105]:
if __name__ == '__main__':
  main()

welcome to 862039649 8 puzzle solver.
Enter your pullze, use a zero to represent the blank(vaild # 0-8)

The choice of algorithm
Enter 1 for Uniform Cost Search
Enter 2 for A* with the Misplaced Tile heurstic
Enter 3 for A* with the Eucledian distance heurstic
Options: Enter 4 for A* with the ma heurstic

start searching.......
[1 0 3]
[4 2 6]
[7 5 8]
operations =  None
depth(g(n)) =  0
h(n) =  3.0
f(n) =  3.0
Expanded nodes =  0 . current queue size =  1
Expanding the nodes..........
Expand 3 nodes

[1 2 3]
[4 0 6]
[7 5 8]
operations =  down
depth(g(n)) =  1
h(n) =  2.0
f(n) =  3.0
Expanded nodes =  3 . current queue size =  2
Expanding the nodes..........
Expand 3 nodes

[1 2 3]
[4 5 6]
[7 0 8]
operations =  down
depth(g(n)) =  2
h(n) =  1.0
f(n) =  3.0
Expanded nodes =  6 . current queue size =  3
Expanding the nodes..........
Expand 2 nodes

[1 2 3]
[4 5 6]
[7 8 0]
operations =  right
depth(g(n)) =  3
h(n) =  0.0
f(n) =  3.0
Expanded nodes =  8 . current queue size =  4
Goal state 

# Reference
### for how to use priority queue
https://www.geeksforgeeks.org/priority-queue-in-pythonn

https://www.educative.io/edpresso/what-is-the-python-priority-queun

### the 6 test cases are get from the \"Project1-Report Guidelines\"
https://drive.google.com/file/d/1h0t0NGQQtI7Fl9B8WzlMLuCn-sFNu0RO/vien
### how to implement a tree in python
https://www.delftstack.com/howto/python/trees-in-pythonn
## euclidean distance formula
https://stackoverflow.com/questions/16869920/a-heuristic-calculation-with-euclidean-distancn